In [10]:
%pip install lightgbm


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score

# --- 데이터 로드 ---
# 캐글에서 다운받은 train.csv, test.csv 파일 경로에 맞게 수정하세요.
train_df = pd.read_csv('../data/input/titanic/train.csv')
test_df = pd.read_csv('../data/input/titanic/test.csv')

# 테스트 데이터의 승객 ID를 저장합니다. (제출 시 사용)
PassengerId = test_df['PassengerId']

# 타겟(정답) 변수와 학습에 사용할 Feature를 분리합니다.
y_train = train_df['Survived']
train_df = train_df.drop('Survived', axis=1)

# 학습 데이터와 테스트 데이터를 합쳐서 일괄 전처리합니다.
data = pd.concat([train_df, test_df], ignore_index=True)
print(f"전체 데이터셋 크기: {data.shape}")

전체 데이터셋 크기: (1309, 11)


In [ ]:
# --- 1. 결측치 및 단순 Feature 처리 ---
# 'Age', 'Fare' 등의 결측치를 중앙값으로 대체
data['Age'] = data['Age'].fillna(data['Age'].median())
data['Fare'] = data['Fare'].fillna(data['Fare'].median())

# 'Embarked' 결측치 최빈값으로 대체
data['Embarked'] = data['Embarked'].fillna(data['Embarked'].mode()[0])

# 'Cabin' (객실 번호): 결측치 여부를 새로운 Feature로 만듭니다.
data['Cabin_Missing'] = data['Cabin'].isna().astype(int)

# --- 2. 범주형 Feature 인코딩 및 Feature Engineering ---
# 'Sex', 'Embarked' 등 범주형 Feature를 Label Encoding으로 변환
# LightGBM은 이를 범주형으로 인식하고 최적으로 처리합니다.

for col in ['Sex', 'Embarked']:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])

# 'Name' Feature에서 '호칭(Title)'을 추출하여 새로운 범주형 Feature로 만듭니다.
data['Title'] = data['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
common_titles = ['Mr', 'Miss', 'Mrs', 'Master']
data['Title'] = data['Title'].apply(lambda x: x if x in common_titles else 'Other')

le_title = LabelEncoder()
data['Title'] = le_title.fit_transform(data['Title'])

# 'FamilySize' (가족 크기) Feature 생성
data['FamilySize'] = data['SibSp'] + data['Parch'] + 1

# 'Ticket', 'Name', 'Cabin' 등 모델에 직접 사용하지 않을 Feature를 제거합니다.
data = data.drop(['Name', 'Ticket', 'Cabin'], axis=1) # axis=0은 가로줄, axis=1은 세로줄

# 학습 데이터와 테스트 데이터를 다시 분리합니다.
X_train = data.iloc[:len(y_train)]
X_test = data.iloc[len(y_train):]

# LightGBM에 범주형 Feature임을 명시할 목록을 지정합니다.
categorical_features = ['Sex', 'Embarked', 'Pclass', 'Title', 'Cabin_Missing']

In [18]:
# --- LightGBM 학습 설정 ---
# 5-Fold 교차 검증을 설정합니다.
N_SPLITS = 5
folds = KFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

# LightGBM 하이퍼파라미터 설정 (타이타닉에 적합한 간단한 설정)
params = {
    'objective': 'binary',            # 이진 분류 문제
    'metric': 'auc',                  # 평가 지표는 ROC-AUC
    'boosting_type': 'gbdt',
    'n_estimators': 1000,             # 부스팅 반복 횟수
    'learning_rate': 0.05,            # 학습 속도
    'num_leaves': 31,                 # 트리의 최대 잎 노드 수
    'max_depth': 5,                   # 트리의 최대 깊이 (과대적합 방지)
    'seed': 42,
    'n_jobs': -1                      # 모든 CPU 코어 사용
}

# --- K-Fold 교차 검증 학습 ---
oof_preds = np.zeros(X_train.shape[0])  # Out-of-Fold 예측 저장
test_preds = np.zeros(X_test.shape[0])  # 테스트 데이터 예측 저장

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
    print(f"--- Fold {fold_+1}/{N_SPLITS} 학습 시작 ---")
    
    # 학습 및 검증 데이터 분리
    X_trn, y_trn = X_train.iloc[trn_idx], y_train.iloc[trn_idx]
    X_val, y_val = X_train.iloc[val_idx], y_train.iloc[val_idx]
    
    # LightGBM 모델 초기화 및 학습
    model = lgb.LGBMClassifier(**params)
    model.fit(X_trn, y_trn,
              eval_set=[(X_val, y_val)],
              eval_metric='auc',
              callbacks=[lgb.early_stopping(100, verbose=False)], # 100회 동안 성능 개선 없으면 조기 종료
              categorical_feature=categorical_features  # 범주형 Feature 명시
             )
    
    # 검증 데이터 예측 (oof: Out-of-Fold)
    oof_preds[val_idx] = model.predict_proba(X_val)[:, 1]
    
    # 테스트 데이터 예측 (모든 Fold 예측 결과를 합산/평균하기 위해 누적)
    test_preds += model.predict_proba(X_test)[:, 1] / N_SPLITS

# --- 최종 성능 평가 ---
final_auc = roc_auc_score(y_train, oof_preds)
print(f"\n--- 5-Fold Cross-Validation 최종 AUC: {final_auc:.4f} ---")

--- Fold 1/5 학습 시작 ---
[LightGBM] [Info] Number of positive: 268, number of negative: 444
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000951 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 453
[LightGBM] [Info] Number of data points in the train set: 712, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.376404 -> initscore=-0.504838
[LightGBM] [Info] Start training from score -0.504838
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positiv

In [23]:
# --- 제출 파일 생성 ---
# 확률 예측 값을 0 또는 1로 변환 (일반적으로 0.5를 기준으로 분류)
submission_predictions = (test_preds > 0.5).astype(int)

submission = pd.DataFrame({
    'PassengerId': PassengerId,
    'Survived': submission_predictions
})

# 결과를 CSV 파일로 저장합니다.
submission.to_csv('../data/output/titanic/titanic_lgbm_submission.csv', index=False)
print("\n제출 파일 'titanic_lgbm_submission.csv' 생성이 완료되었습니다.")


제출 파일 'titanic_lgbm_submission.csv' 생성이 완료되었습니다.


# 임시

In [ ]:
from sklearn.preprocessing import LabelEncoder

# 1. 데이터 준비
items = ['TV', '냉장고', '전자레인지', '컴퓨터', '선풍기', '선풍기', '믹서', '믹서']

# 2. LabelEncoder 객체 생성
encoder = LabelEncoder()

# 3. fit()과 transform()으로 변환 (혹은 fit_transform() 한번에 사용)
encoder.fit(items) # 데이터의 고유한 값들을 찾아 번호를 매길 준비를 합니다.
labels = encoder.transform(items) # 실제 데이터를 숫자로 바꿉니다.

print('인코딩 변환값:', labels)
print('인코딩 클래스:', encoder.classes_)

# 4. 원본으로 되돌리기 (디코딩)
originals = encoder.inverse_transform([4, 0, 1])
print('디코딩 원본값:', originals)

인코딩 변환값: [0 1 4 5 3 3 2 2]
인코딩 클래스: ['TV' '냉장고' '믹서' '선풍기' '전자레인지' '컴퓨터']
디코딩 원본값: ['전자레인지' 'TV' '냉장고']
